# Проект для «Викишоп»

## Подготовка

In [1]:
# импортируем библиотеки
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
import re
from nltk.stem import WordNetLemmatizer 

In [2]:
# читаем файл
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет. Типы данных в порядке. Подготовим корпус для обучения моделей.

In [5]:
corpus = data['text'].apply(lambda x: x.lower())

In [6]:
# разобьем на токены и уберем лишние пробелы
def token(corpus):
    x = re.sub(r"[^'a-z]", " ", corpus)
    return x.split()
    


In [7]:
tokens = corpus.apply(token)
tokens

0         [explanation, why, the, edits, made, under, my...
1         [d'aww, he, matches, this, background, colour,...
2         [hey, man, i'm, really, not, trying, to, edit,...
3         [more, i, can't, make, any, real, suggestions,...
4         [you, sir, are, my, hero, any, chance, you, re...
                                ...                        
159566    [and, for, the, second, time, of, asking, when...
159567    [you, should, be, ashamed, of, yourself, that,...
159568    [spitzer, umm, theres, no, actual, article, fo...
159569    [and, it, looks, like, it, was, actually, you,...
159570    [and, i, really, don't, think, you, understand...
Name: text, Length: 159571, dtype: object

In [8]:
# выделим леммы
nltk.download('wordnet')
m = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def lemma(tokens):
    lem = []
    for token in tokens:
        lem.append(m.lemmatize(token))
        
    return " ".join(lem)

In [10]:
data['lemmas'] = tokens.apply(lemma)

In [11]:
data['lemmas']

0         explanation why the edits made under my userna...
1         d'aww he match this background colour i'm seem...
2         hey man i'm really not trying to edit war it's...
3         more i can't make any real suggestion on impro...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself that is a ho...
159568    spitzer umm there no actual article for prosti...
159569    and it look like it wa actually you who put on...
159570    and i really don't think you understand i came...
Name: lemmas, Length: 159571, dtype: object

In [12]:
#делим данные на тестовую и обучающую выборки

features_train, features_test, target_train, target_test = train_test_split(data['lemmas'], data['toxic'], random_state=10, test_size=0.2)


In [13]:
# создадим счетчик для построение tfidf матрицы
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# создадим матрицу для обучающей и тестовой выборки
# посмотрим на результат модели логистической регрессии
tdidf_train = count_tf_idf.fit_transform(features_train)

tdidf_test = count_tf_idf.transform(features_test)

Данные преобразованы в матричный вид, учитывающий важность отдельных слов методикой TD-IDF. Далее обучим несколько моделей на основе новых данных.

## Обучение

In [15]:
# постороим модель логистической регрессии
logistic_m = LogisticRegression(random_state=10, solver ='liblinear', class_weight = 'balanced')

logistic_m.fit(tdidf_train, target_train)

LogisticRegression(class_weight='balanced', random_state=10, solver='liblinear')

In [17]:
# постороим модель случайного леса
forest_m = RandomForestClassifier(random_state=10, max_depth=5, n_estimators=60)

forest_m.fit(tdidf_train, target_train)

RandomForestClassifier(max_depth=5, n_estimators=60, random_state=10)

In [18]:
# постороим модель градиентного бустинга
boost_m = CatBoostClassifier(iterations=1000,  eval_metric='F1', random_state=10, max_depth=6)

boost_m.fit(tdidf_train, target_train, verbose=50)

Learning rate set to 0.081698
0:	learn: 0.4522286	total: 3.19s	remaining: 53m 4s
50:	learn: 0.5796465	total: 2m 8s	remaining: 39m 52s
100:	learn: 0.6221924	total: 4m 10s	remaining: 37m 12s
150:	learn: 0.6668982	total: 6m 6s	remaining: 34m 20s
200:	learn: 0.6858509	total: 8m 6s	remaining: 32m 15s
250:	learn: 0.7064334	total: 10m 13s	remaining: 30m 30s
300:	learn: 0.7218890	total: 12m 3s	remaining: 28m
350:	learn: 0.7319014	total: 13m 58s	remaining: 25m 51s
400:	learn: 0.7434555	total: 15m 51s	remaining: 23m 41s
450:	learn: 0.7486162	total: 17m 41s	remaining: 21m 32s
500:	learn: 0.7568162	total: 19m 32s	remaining: 19m 27s
550:	learn: 0.7608914	total: 21m 29s	remaining: 17m 30s
600:	learn: 0.7645185	total: 23m 19s	remaining: 15m 29s
650:	learn: 0.7672664	total: 25m 12s	remaining: 13m 30s
700:	learn: 0.7699905	total: 27m 6s	remaining: 11m 33s
750:	learn: 0.7746943	total: 28m 58s	remaining: 9m 36s
800:	learn: 0.7806958	total: 30m 57s	remaining: 7m 41s
850:	learn: 0.7845803	total: 32m 49s	re

## Выводы

In [19]:
# посмотрим на результат модели логистической регрессии
logistic_pred = logistic_m.predict(tdidf_test)
print('f1_score', f1_score(target_test, logistic_pred))

f1_score 0.7562465297057189


Модель логистической регрессии смогла успешно справиться с поставленной задачей  достичь заданной метрики. Посмотрим на другие модели.

In [20]:
# посмотрим на результат модели леса
forest_pred = forest_m.predict(tdidf_test)
print('f1_score', f1_score(target_test, forest_pred))

f1_score 0.0


Модель слуйчаного леса оказалась полностью  бесполезна для решения данной задачи. она пресдказала только одни 0, тем самым опустив свою метрику полноты до 0 и в свою очередь метрику f1. Возможно, для этой модели можно было бы увеличить количество положительных результатов, чтобы лучше ее научить, но она явно уступает другим моделям.

In [21]:
# посмотрим на результат catboost
boost_pred = boost_m.predict(tdidf_test)
print('f1_score', f1_score(target_test, boost_pred))

f1_score 0.761835130689088


Эта модель смогла добиться лучше результата незначительно обогнав модель логистической регрессии.

In [21]:
#Сведем итоговую мини-табличку

index = ['Логистическая Регрессия', 'Случайный лес', 'Catboost']

results = 0.756, 0, 0.762

display(pd.DataFrame(results, columns=['f1_score'], index=index))

,f1_score
Логистическая Регрессия,0.726
Случайный лес,0.000
Catboost,0.742


Для решения даннной задачи по поиску токсичных комментариев - лучшей моделью по результатам метрики стала модель градиентного бустинга. Однако стоит учесть, что скорость обучения у этой модели в разы ниже, чем у модели логистической регрессии. Поэтому если скорость является сильно подовляющим фактором, то явным чемпионом здесь будет логистическая регрессия.